In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\megam\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as  plt 

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df = df[df["koi_disposition"] != 'CANDIDATE']
df["koi_disposition"].value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
Name: koi_disposition, dtype: int64

# Select your features (columns)

In [8]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_steff', 'koi_slogg', 'koi_srad']]
selected_features.koi_period.astype(float)

0       54.418383
1       19.899140
2        1.736952
3        2.525592
4        4.134435
          ...    
6983    21.513523
6986     8.589871
6987     0.527699
6989     0.681402
6990     4.856035
Name: koi_period, Length: 5304, dtype: float64

# Create a Train Test Split

Use `koi_disposition` for the y values

In [10]:
selected_features['koi_disposition_dummy'] = selected_features.koi_disposition.map({'FALSE POSITIVE':0, 'CONFIRMED':1})
y = selected_features['koi_disposition_dummy']

AttributeError: 'DataFrame' object has no attribute 'koi_disposition'

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=0)

In [7]:
y_train.head()

5058    0
5458    0
6589    0
5315    0
781     1
Name: koi_disposition_dummy, dtype: int64

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
minmax_fitted = minmax.fit(X_train)
X_trains = minmax_fitted.transform(X_train)
X_tests = minmax_fitted.transform(X_test)
# y_trains = minmax.fit_transform(y_train)
# y_tests = minmax.fit_transform(y_test)

# Train the Model



## Not using scaled

In [17]:
from sklearn.cluster import KMeans
from sklearn import metrics
model1 = KMeans(n_clusters=3)

In [18]:
model1.fit(X_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [19]:
# Predict the clusters
pred = model1.predict(X_test)
pred_train = model1.predict(X_train)

In [25]:
# testing score
score = metrics.f1_score(y_test, pred, pos_label=list(set(y_test)), average=None)
# training score
score_train = metrics.f1_score(y_train, pred_train, pos_label=list(set(y_train)), average=None)

print(f"Training Data Score: {score_train[0]}")
print("-----------------------------------------------------")
print(f"Testing Data Score: {score[0]}")

Training Data Score: 0.7941800545619885
-----------------------------------------------------
Testing Data Score: 0.7972789115646258


## Using scaled

In [15]:
kmeansS = KMeans(n_clusters=3)
kmeansS.fit(X_trains)

# Predict the clusters
preds = kmeansS.predict(X_tests)
pred_trains = kmeansS.predict(X_trains)

# testing score
scores = metrics.f1_score(y_test, preds, pos_label=list(set(y_test)), average=None)
# training score
score_trains = metrics.f1_score(y_train, pred_trains, pos_label=list(set(y_train)), average=None)

print(f"Testing score: {kmeansS.score(X_tests, y_test)}")
print(f"Training score: {kmeansS.score(X_trains, y_train)}")

print(scores)
print(score_trains)

Testing score: -29.14663974446135
Training score: -87.19826994353079
[0.21327968 0.5829904  0.        ]
[0.23431242 0.59401416 0.        ]


C:\Users\megam\Anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\classification.py:1259: UserWarning: Note that pos_label (set to [0, 1]) is ignored when average != 'binary' (got None). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
C:\Users\megam\Anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Save the Model

In [26]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'megan_okerlund_model1.sav'
joblib.dump(model1, filename)

['megan_okerlund_model1.sav']